# Swedish Lighthouses Challenge


**SkyNet** (ghost name) is scoping out a project to aid the search and rescue operations of the Swedish Coast Guard. Their goal is to establish a network of docked drones along the coastline that provide a rapid response to emergency transmissions.

(The SkyNet Dock is a cloud-connected, weatherproof home base and charging station that enables our drones to fly without a pilot on-scene.)

The customer is interested in establishing coverage from their main office in Stockholm east to the island of Vindö and then southwest to Torö.

They have permission to build a dock into any lighthouse in the region. They are interested in building a route such that a drone can hop from dock to dock from Stockholm to Torö, so that they can recover their fleet and swap drones without sending humans to the lighthouses.

**Guidelines** - This challenge tests the ability to write quality code, interact with APIs, visualize data, and employ algorithms to solve problems. We score submissions by the clarity of the code, visuals, and written documentation. We do not score by the amount of time spent, so go as deep as you like, or if you are short on time then take a simpler approach to Part 2 and 3.

<img width="500px" src="https://i.imgur.com/5H88NQ3.png"/>

# Part 1: Find all of the lighthouses

Use [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap) to find known lighthouses in the region and get that data into your notebook. OSM provides APIs to query their geographic database by structure type. Plot the lighthouses on a map using [pydeck](https://deckgl.readthedocs.io/en/latest/).

Note that you are free to install python packages in colab: `!pip install pydeck`

In [ ]:
%pip install geocoder
%pip install pydeck
%pip install overpy # forming the overpass string-shaped request to geocoder
%pip install geopy

In [ ]:
import geocoder

class gpsLocation():
  """
  encapsulates the lattitude, longitude pairs, and reversed notation for distance
  computations
  """
  def __init__(self, lat_lon):
    self.lat_lon = lat_lon
    self.lon_lat = lat_lon[::-1]

  def swap_lat_long(self):
    self.lon_lat[0], self.lon_lat[1] = self.lat_lon[1], self.lat_lon[0]

  def get_lat_lon(self):
    return self.lat_lon

  def get_lon_lat(self):
      return self.lon_lat

class gpsLocations:
  """
  Encapsulates a list of coordinates
  """
  def __init__(self):
    self.lat_lon = []
    self.lon_lat = []
    self.points = 0

  def update(self,lat,lon):
      self.lat_lon.append((lat,lon))
      self.lon_lat.append((lon,lat))

  def add(self,data):
    """
    populate from data retrieved from Open Street Map
    """
    for element in data['elements']:
      if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        self.update(lat,lon)
      elif 'center' in element:
        lon = element['center']['lon']
        lat = element['center']['lat']
        self.update(lat,lon)
      self.points = len(self.lat_lon)

  def addList(self, lat_lon_points):
    """
    appends a list of points to the object
    """
    for p in lat_lon_points:
      self.update(p[0],p[1])
    self.points = len(self.lat_lon)

  def get_lat_lon(self):
    return self.lat_lon

  def get_lon_lat(self):
    return self.lon_lat

  def npoints(self):
    return self.points

def get_city_coords(cityState):
  """
  gets the coordinates of the requested cities from osm
  """
  g = geocoder.osm(cityState)
  # have lon lat from osm call
  lat_lon_coords = gpsLocation([g.osm['y'],g.osm['x']])
  return gpsLocation([g.osm['y'],g.osm['x']])




# Part 2: Choose where to build docks

Suggest a minimal set of lighthouses at which to build a dock that would allow a drone to traverse from Stockholm to Torö, if each drone has a usable battery life of 30 minutes and travels at a ground speed of 12 meters per second. Plot the route, assuming approximate endpoints at Stockholm and Torö. Also print the total route distance.

It's okay to make reasonable assumptions here and state your thinking. You can go as straightforward or complex as you like.

In [ ]:
# global variables
Stockholm = get_city_coords("Stockholm, Sweden")
Vindo = get_city_coords("Vindo, Sweden")
Toro = get_city_coords("gabrielstorp, Sweden") # there is a different location Toro in database
Cities = gpsLocations()
Cities.addList([Stockholm.get_lat_lon(),Vindo.get_lat_lon(),Toro.get_lat_lon()])
vel = 12 # drone velocity m/s
life = 30 # battery life in minutes
max_range = life*60*vel/1000 *.9 # max range is chosen as 90% of the abslute range
print(f'drone max_range (km): {max_range}')

drone max_range (km): 19.44


In [ ]:
# build query to return gps locations of lighthouses
import requests
import json
def get_bounded_lighthouses(lower_left: list[float], upper_right: list[float]) -> str:
  """
  find all lighthouses within a bounded box region
  lower_left: [lat, lon] coordinates of lower left of boxed area of interest
  upper_right: [lat, lon] coordinates of upper right of boxed area of interest\
  returns: list of lighthouses within bounded region
  """
  overpass_url = "http://overpass-api.de/api/interpreter"

  bounds = f"{lower_left[0]:.5f},{lower_left[1]:.5f},{upper_right[0]:.5f},{upper_right[1]:.5f}"

  query = """
  [out:json];
  area["ISO3166-1"="SE"][admin_level=2];
  (node["man_made"="lighthouse"](%s);
  );
  out center;
  """ % bounds
  response = requests.get(overpass_url,
                          params={'data': query})
  return response.json()


In [ ]:
from collections import defaultdict

class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self.edges = defaultdict(list)
        self.weights = {}

    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

In [ ]:
def dijsktra_search(graph: Graph, anchor: tuple, target: tuple) -> (list, float):
    """
    performs the dijsktra search algorithm on a weighted graph
    graph: weighted graph of all lighthouses and neighbors within range
    anchor: (lat,lon) starting city from which to find the shortest path to target
    target: (lat,lon) ending city we want the optimal path to get to from anchor
    returns: list of lat/lon lighthouses between cities, optimal distance
    """
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {anchor: (None, 0)}
    current_node = anchor
    visited = set()

    while current_node != target:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)

        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}

        if not next_destinations:
            return "Route Not Possible",0
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])

    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return path, shortest_paths[path[-1]][1]

In [ ]:
import copy
from geopy import distance
import numpy as np
def lighthouses_in_range(anchor: gpsLocation, lighthouses: json, max_range: int) -> (list,list):
  """
  returns all the lighthouses within a max range of the anchor
  anchor: starting location from which to search for lighthouses within range
  lighthouses: json string containing lat/lon for all lighthouses in bounding box
  max_range: maximum distance between anchor and lighthouses for drone to travel
  returns: list (lat,lon) of all lighthouses within range of the anchor
  """
  nodes = []
  distances = []
  for i in range(len(lighthouses['elements'])):
    (lat,lon) = (lighthouses['elements'][i]['lat'],lighthouses['elements'][i]['lon'])
    d = distance.distance(anchor.get_lat_lon(),[lat,lon])
    if d.km < max_range:
      nodes.append([lat,lon])
      distances.append(d)
  idx = np.argsort(distances)
  orig_distances = copy.deepcopy(distances)
  distances.sort()
  distances = [d for d in distances if d.km > 0 and d.km < max_range]
  valid_nodes = []
  for i in idx:
    valid_nodes.append(nodes[i])
  return valid_nodes, distances

In [ ]:
# add edges between a root
def add_edges(root: gpsLocation, lighthouses: json, max_range: float, graph: Graph) -> Graph:
  """
  Given the anchor location, lighthouse locations and range, build the graph
  root: starting location to build a list of all lighthouses within range
  lighthouses: json string of all the lighthouse locations
  max_range: distance in km between root and valid nodes
  graph: the data structure containing all the nodes and allowed neighbors
  returns: graph
  """
  nodes, weights = lighthouses_in_range(root,lighthouses,max_range)
  for n, w in zip(nodes, weights):
    if w.km > 0 and w.km < max_range:
      edge = (tuple(root.get_lat_lon()), tuple(n), w.km)
      graph.add_edge(*edge)
  return graph

In [ ]:
# build graph for leg 1, Stockholm - Vindo
def build_graph(root: gpsLocation, target: gpsLocation, lighthouses: json) -> Graph:
  """
  build the graph for root, target pair and valid lighthouse stations
  root: starting node, will find the optimal path from this to target
  target: ending node
  lighthouses: json string containing lat/lon for all valid lighthouses
  returns: graph containing all nodes and neighbors within bounds
  """
  graph = Graph()
  # add anchor cities and neighbors within range to graph
  add_edges(root,lighthouses,max_range,graph)
  add_edges(target,lighthouses,max_range,graph)
  # add rest of valid lighthouse locations to graph
  for i in range(len(lighthouses['elements'])):
      (lat,lon) = (lighthouses['elements'][i]['lat'],lighthouses['elements'][i]['lon'])
      graph = add_edges(gpsLocation([lat,lon]),lighthouses, max_range, graph)
  return graph

In [ ]:
def valid_lighthouses(root: gpsLocations, target: gpsLocations, margin: float) -> list:
  """
  pulls the allowed lighthouse locations from the database
  root: starting location from which to find optimal path to target
  target: ending location
  margin: how much to extend the bounding box region containing valid lighthouses
  returns: valid lighthouses found between a rectangular region containing the
    root and target locations as the opposite corners of the rectangle
  """
  lighthouses = gpsLocations()
  lower_left = root.get_lat_lon()
  lower_left[0] -= margin # add bit of margin
  upper_right = target.get_lat_lon()
  upper_right[0] += margin # add bit of margin
  lighthouses = get_bounded_lighthouses(lower_left, upper_right)
  return lighthouses


In [ ]:
def find_best_path(root: gpsLocation, target: gpsLocation, graph: Graph) -> (list, float):
  """
  use dijsktra's algorithm to find the optimal path between root and target
  root: starting city to find optimal path to the target city from
  target: final city to plan route towards from target city
  graph: data structure with the cities/lighthouses and valid neighbors to search
  returns: shortest route between recharging stations to go from root city to target,
    distance of path taken between root and target
  """
  best_path, weight = dijsktra_search(graph,tuple(root.get_lat_lon()),tuple(target.get_lat_lon()))
  path = []
  path.append(root.get_lon_lat())
  for b in best_path[1:-1]:
    path.append(gpsLocation(b).get_lon_lat())
  # strange thing with the endpoints getting corrupted upon the get_lon_lat()
  path.append(target.get_lon_lat())
  return path, weight


# Part 2: Choose where to build docks

Suggest a minimal set of lighthouses at which to build a dock that would allow a drone to traverse from Stockholm to Torö, if each drone has a usable battery life of 30 minutes and travels at a ground speed of 12 meters per second. Plot the route, assuming approximate endpoints at Stockholm and Torö. Also print the total route distance.

It's okay to make reasonable assumptions here and state your thinking. You can go as straightforward or complex as you like.

In [ ]:
stockholm_vindo_path = [] # gps lat/lon of all stops between Stockholm and Vindo
vindo_toro_path = [] # gps lat/lon of all stops between Vindo and Toro
# 1st leg between Stockholm and Vindo
fudge = .05 # how much to extend the bounding box from location of city in degrees
# get lighthouses within bounding box with Stockholm and Vindo as corners of the box
stockholm_vindo_houses = valid_lighthouses(Stockholm,Vindo,fudge)
# build the graph between the cities and the valid lighthouses
graph = build_graph(Stockholm, Vindo, stockholm_vindo_houses)
# compute the optimal path between Stockholm and Vindo
stockholm_vindo_path, stockholm_vindo_distance = find_best_path(Stockholm, Vindo, graph)
# 2nd leg between Toro and Vindo
fudge = 0 # how much to extend the bounding box from location of city in degrees
# get lighthouses within bounding box with Toro and Vindo as corners of the box
vindo_toro_houses = valid_lighthouses(Toro,Vindo,fudge)
# build the graph between the cities and the valid lighthouses
graph = build_graph(Toro, Vindo, vindo_toro_houses)
# compute the optimal path between Stockholm and Vindo
vindo_toro_path, toro_vindo_distance = find_best_path(Toro, Vindo, graph)

In [ ]:
# format all lighthouses along the way for plotting
lighthouses = []
for s in stockholm_vindo_houses['elements']:
  lighthouses.append((s['lon'],s['lat']))
for s in vindo_toro_houses['elements']:
  lighthouses.append((s['lon'],s['lat']))

In [ ]:
import pydeck as pdk

INITIAL_VIEW_STATE = pdk.ViewState(
  latitude=Stockholm.get_lat_lon()[0],
  longitude=Stockholm.get_lat_lon()[1],
  zoom=8,
  max_zoom=16,
  pitch=0,
  bearing=0
)

house_layer = pdk.Layer(
  'ScatterplotLayer',  # `type` positional argument is here
  lighthouses,
  get_position='-',
  auto_highlight=True,
  get_radius=1000,
  get_fill_color='[80, 200, 200, 140]',
  pickable=True)

path_layer =  pdk.Layer(
  'ScatterplotLayer',  # `type` positional argument is here
  stockholm_vindo_path,
  get_position='-',
  auto_highlight=True,
  get_radius=2000,
  get_fill_color='[280, 10, 200, 140]',
  pickable=True)

path_layer2 =  pdk.Layer(
  'ScatterplotLayer',  # `type` positional argument is here
  vindo_toro_path,
  get_position='-',
  auto_highlight=True,
  get_radius=2000,
  get_fill_color='[280, 10, 200, 140]',
  pickable=True)

city_layer = pdk.Layer(
  'ScatterplotLayer',  # `type` positional argument is here
  Cities.get_lon_lat(),
  get_position='-',
  auto_highlight=True,
  get_radius=1000,
  get_fill_color='[180, 200, 200, 140]',
  pickable=True)
# Render
r = pdk.Deck(layers=[house_layer, city_layer,path_layer,path_layer2], initial_view_state=INITIAL_VIEW_STATE, tooltip={"text": "{name}\n{address}"})
r.to_html("scatterplot_layer.html")

<IPython.core.display.Javascript object>

## Above we have plotted the cities (gray), optimal locations for docking stations at lighthouses (pink) and all lighthouses (aqua)

# Part 3: Get drones from Stockholm to the docks!

What is the difference between the straight line path and the chosen dock path total distances between cities?

In [ ]:

direct_path = distance.distance(Stockholm.get_lat_lon(),Vindo.get_lat_lon())
print(f'stockholm-vindo distance: {direct_path.km} via stations: {stockholm_vindo_distance}')

direct_path = distance.distance(Vindo.get_lat_lon(),Toro.get_lat_lon())
print(f'toro-vindo distance: {direct_path.km} via stations: {toro_vindo_distance}')


# your estimate of the duration required to fill the docks

stockholm-vindo distance: 38.19709785638198 via stations: 38.25241609033256
toro-vindo distance: 80.47329158499772 via stations: 84.53695610659824
